---
title: "Public water supply"
---

In [ ]:
from pathlib import Path

from ribasim import Model, Node
from ribasim.nodes import (
    user_demand,
)
from shapely.geometry import Point

In [ ]:
base_dir = Path("c:/bin/ribasim")
model_dir = base_dir / "Crystal_Basin"
data_path = model_dir / "data/input/ACTINFLW.csv"

starttime = "2022-01-01"
endtime = "2023-01-01"
model = Model(
    starttime=starttime,
    endtime=endtime,
    crs="EPSG:4326",
)


## Public Water Supply

### Rename the saving files
Rename the files to `Crystal_2.2`

### Add a demand node
$50.000$ people live in Crystal City.
To represents the total flow rate or abstraction rate required to meet the water demand of $50.000$ people, another demand node needs to be added assuming a return flow of $60%$.

In [ ]:
model.user_demand.add(
    Node(9, Point(0.0, -0.25), name="PWS"),
    [
        user_demand.Time(
            # Total demand in m³/s
            demand=[
                0.07,
                0.08,
                0.09,
                0.10,
                0.12,
                0.14,
                0.15,
                0.14,
                0.12,
                0.10,
                0.09,
                0.08,
            ],
            return_factor=0.6,
            min_level=0,
            priority=1,
            time=[
                starttime,
                "2022-02-01",
                "2022-03-01",
                "2022-04-01",
                "2022-05-01",
                "2022-06-01",
                "2022-07-01",
                "2022-08-01",
                "2022-09-01",
                "2022-10-01",
                "2022-11-01",
                "2022-12-01",
            ],
        )
    ],
)

### Add the edges
The connection between the reservoir and the demand node needs to be defined:

In [ ]:
model.edge.add(model.flow_boundary[1], model.basin[3])
model.edge.add(model.flow_boundary[2], model.basin[4])
model.edge.add(model.basin[3], model.user_demand[6])
model.edge.add(model.basin[3], model.user_demand[9])
model.edge.add(model.user_demand[6], model.basin[4])
model.edge.add(model.user_demand[9], model.basin[4])
model.edge.add(model.basin[3], model.tabulated_rating_curve[7])
model.edge.add(model.tabulated_rating_curve[7], model.basin[4])
model.edge.add(model.basin[4], model.tabulated_rating_curve[5])
model.edge.add(model.tabulated_rating_curve[5], model.terminal[8])

### Adjust the name dictionaries

In [ ]:
# Dictionary mapping node_ids to names
edge_names = {
    (1, 3): "Main",
    (2, 4): "Minor",
    (3, 6): "IrrA Demand",
    (6, 4): "IrrA Drain",
    (3, 9): "PWS Demand",
    (9, 4): "PWS Return",
    (3, 7): "Rsv2Main",
    (7, 4): "Main2Conf",
    (4, 5): "Conf2TRC",
    (5, 8): "TRC2Term",
}

### Check the simulated demands
@fig-sim8 shows the flow to (PWS Demand) and out (PWS Return) of the PWS node.
@fig-sim9 shows the downstream flow to the ocean.
The impact is clear.
Due to the demands upstream (irrigation and public water supply) an expected decrease of discharge is shown downstream.

![Simulated flows to and from the city](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-flows-to-and-from-the-city.png){fig-align="left" #fig-sim8}

![Simulated basin storages and levels](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-storages-and-levels-reservoir.png){fig-align="left" #fig-sim9}